#Data Augmentation
Here, the aim is to augment the images in the dataset in order to increase its size. The transformations used are : horizontal flip, zoom range of 0.04, height shift range of 0.04, width shift range of 0.04, shear range of 0.04. The data augmentation is done with the help of Image Data Generator.

# Authentication

In [ ]:
#Authentication of user 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
#This cell is used to create a folder 'drive' and load the files of drive in it
!mkdir drive
!google-drive-ocamlfuse drive #ocaml fuse is defined in the previous cell
!ls drive


 ADNI_dataset_2000	       CN
 ADNI_dataset_450	      'Colab Notebooks'
 ADNI_dataset_splitted_2000   'Data Augmentation'
 ADNI_dataset_splitted_30000   Shitty_Augmented
 ADNI_dataset_splitted_450    'Transfer-Learning in Keras.ipynb'
 ADNI_dataset_splitted_5000


# *Data Augmentation*


In [ ]:
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

In [ ]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
import os

# load the input image, convert it to a NumPy array, and then
# reshape it to have an extra dimension
print("[INFO] loading example image...")
main_path = '/content/drive/Dataset/AD'
adni_list = os.listdir(main_path)
for infile in adni_list:
  print('Next image...',infile)
  image = load_img(os.path.join(main_path+'/'+infile))
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)
  # construct the image generator for data augmentation then
  # initialize the total number of images generated thus far
  aug = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      zoom_range=0.05,
      width_shift_range=0.05,
      height_shift_range=0.05,
      shear_range=0.05,
      horizontal_flip=True,
      fill_mode="nearest")
  
  total = 0
  # construct the actual Python generator
  print("[INFO] generating images...")
  imageGen = aug.flow(image,batch_size=1, save_to_dir='directory_in_which_augmented_images_are_saved',
	save_prefix="recent_augmented_", save_format="png")
  # loop over examples from our image data augmentation generator
  for image in imageGen:
    # increment our counter
    total += 1
    # if we have reached the specified number of examples, break
    # from the loop
    if total == 5:
      print('This loop done')
      break

Using TensorFlow backend.


[INFO] loading example image...
Next image... 10671.png
[INFO] generating images...
This loop done
Next image... 9873.png
[INFO] generating images...
This loop done
Next image... 1288.png
[INFO] generating images...
This loop done
Next image... 49090.png
[INFO] generating images...
This loop done
Next image... 8700.png
[INFO] generating images...
This loop done
Next image... 34912.png
[INFO] generating images...
This loop done
Next image... 49012.png
[INFO] generating images...
This loop done
Next image... 485.png
[INFO] generating images...
This loop done
Next image... 10248.png
[INFO] generating images...
This loop done
Next image... 7222.png
[INFO] generating images...
This loop done
Next image... 70064.png
[INFO] generating images...
This loop done
Next image... 11010.png
[INFO] generating images...
This loop done
Next image... 4176.png
[INFO] generating images...
This loop done
Next image... 14095.png
[INFO] generating images...
This loop done
Next image... 20422.png
[INFO] genera